# Exercise04 : トレーニングパイプラインの作成

AML 機械学習パイプラインを利用することで、以下のような目的で ML ワークフローを作成することができます。

- MLOps 統合のための再トレーニングパイプラインを構築することができます。
- [Exercise03 : Web サービスとして公開する](./exercise03_publish_model.ipynb) において、リアルタイムスコアリングの代わりにバッチスコアリングのパイプラインを構築することができます。

> Note : CI/CD ツールと連携したリファレンスアーキテクチャについては、[こちら](https://docs.microsoft.com/azure/architecture/reference-architectures/ai/mlops-python)をご参照ください。

ML パイプラインは、以下の方法で呼び出すことができます。

- タイムベースのスケジュール呼び出し
- 公開されたエンドポイント（REST）によるオンデマンドな呼び出し
- ファイル変更などの複合イベントによるトリガーベースの呼び出し（Azure Event Grid や Azure Logic Apps などによる）

この演習では、トップレベル（パイプラインの）出力にモデルメトリクスを返す、シンプルなトレーニングパイプラインを作成します。


## 1. コンピュート クラスターの作成

パイプラインを実行するための新しい AML コンピュート クラスターを作成します。<br>
すでに作成済みの場合は、以前のクラスターを使いまわします。

パイプラインが起動されると、このコンピュート クラスターも起動されます。パイプラインが終了すると、このコンピュート クラスターも自動的にゼロにスケールダウンされます。

In [ ]:
"""
!az ml compute create --name demo-cpucluster1 \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 1 \
  --size Standard_D2_v2 \
  --idle-time-before-scale-down 1800
"""

## 2. トレーニングスクリプトをファイルとして保存する (train.py)

Exercise02 で作成したトレーニングスクリプト `/scripts/train.py` を再利用します。

## 3. モデル登録スクリプトをファイルとして保存する (register.py)

この例では、モデルの学習、モデル登録のためのパイプラインを作成します。
このパイプラインでは、以下の手順が実行されます。

1. モデルの学習が行われる。
2. モデルを登録する。モデルメトリクスをパイプラインの出力として設定する。

そして、各ソースコードは以下のように保存されます。

- training script ```./scripts/train.py```
- register script ```./scripts/register.py```

モデル名（モデルディレクトリのサブフォルダ名）は、モデル情報ファイル（JSONテキスト）に保存され、次のステップに渡されます。

In [ ]:
%%writefile scripts/register.py

import argparse
import json
import mlflow
from mlflow.pyfunc import load_model
from mlflow.tracking import MlflowClient

def parse_args():

    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', type=str, help='Name under which model will be registered')
    parser.add_argument('--model_path', type=str, help='Model directory')
    parser.add_argument('--deploy_flag', type=str, help='A deploy flag whether to deploy or no')

    args, _ = parser.parse_known_args()
    print(f'Arguments: {args}')

    return args


def main():

    args = parse_args()

    model_name = args.model_name
    model_path = args.model_path
    deploy_flag = int(args.deploy_flag)

    mlflow.set_tag("model_name", model_name)
    mlflow.set_tag("model_path", model_path)
    mlflow.set_tag("deploy_flag", int(deploy_flag))

    # For information transfer between pipelines
    json_open = open(model_path + "/metric.json", 'r')
    json_load = json.load(json_open)
    rmse = json_load["RMSE"]
    r2 = json_load["R2"]
    print("rmse: " + str(rmse))
    print("r2: " + str(r2))

    mlflow.set_tag("RMSE", rmse)
    mlflow.set_tag("R2", r2)

    if deploy_flag==1:
        # Load model from model_path
        model = load_model(model_path + "/models") 
        # Log the sklearn model and register as version 1
        modelreg = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="models",
            registered_model_name=model_name
        )
        print(modelreg)

        #Since log_model cannot register tags in models, use set_model_version_tag instead.
        client = MlflowClient() 
        model_info = client.get_registered_model(model_name)
        model_version = model_info.latest_versions[0].version
        dict_metrics =  {"RMSE": rmse, "R2": r2}
        client.set_model_version_tag(model_name, str(model_version), "metrics", json.dumps(dict_metrics))
        print("Model registered!")
    else:
        print("Model will not be registered!")

if __name__ == "__main__":
    main()




## 4. MLパイプラインの構築と実行

yaml でパイプラインを構成し、生成されたパイプラインに対してジョブを投入してみましょう。

> 注意 : この例では、 ``diabetes_data_oh4ml_350records`` という名前の登録済みデータ アセットをコンピュートターゲットにマウントして使用します。データ アセット準備のため、"[Exercise00-2 : Prepare Data](./exercise00_1_prepare_data.ipynb)" を実行します。

In [ ]:
%%writefile 04_training_pipeline_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
experiment_name: 04_pipeline_oh4ml_2step
description: 2-Step Pipeline Example (Train, Register)

# <inputs_and_outputs>
inputs:
  input: 
    type: uri_file
    #path: azureml:diabetes_data_oh4ml@latest
    path: azureml:diabetes_data_oh4ml_350records@latest
  model_name: diabetes_model_oh4ml
outputs: 
  outputs:

settings:
  default_datastore: azureml:workspaceblobstore
  default_compute: azureml:demo-cpucluster1
  continue_on_step_failure: false

jobs:
  train_model:
    name: train_model
    display_name: train-model
    code: ./scripts
    command: >-
      python train.py 
      --input_data ${{inputs.diabetes_data}} 
      --output_dir ${{outputs.model_output}}
    environment: azureml:diabetes-env@latest
    inputs:
      diabetes_data: ${{parent.inputs.input}}
    outputs:
      model_output: ${{parent.outputs.outputs}}

  register_model:
    name: register_model
    display_name: register-model
    code: ./scripts
    command: >-
      python register.py 
      --model_name ${{inputs.model_name}} 
      --model_path ${{inputs.model_path}} 
      --deploy_flag ${{inputs.deploy_flag}}
    environment: azureml:diabetes-env@latest
    inputs:
      model_name: ${{parent.inputs.model_name}}
      model_path: ${{parent.jobs.train_model.outputs.model_output}}
      deploy_flag: 1

このパイプラインを実行するためのジョブを投入します。

In [ ]:
!az ml job create --file 04_training_pipeline_job.yml

AML studio UI にアクセスし、パイプラインの結果をジョブで確認します。(下記参照)

![Pipeline results](../images/005.png)

パイプラインの出力でモデルメトリクスを抽出することができます。<br>
このトレーニングパイプラインがパスされた場合、MLOps 統合の次のステージを呼び出すことができます。

## 5.  コンピュート クラスターの削除

AML コンピュート クラスターは非アクティブになると自動的にノードが終了するので、コスト節約のために削除する必要はありません。しかし、もしクリーンアップしたい場合は、以下のように実行してください。

In [ ]:
"""
!az ml compute delete --name demo-cpucluster1 \
  --yes
"""